In [34]:
# Usuals imports 
import numpy as np
import pandas as pd
from time import sleep
import re

## Informations about previous files

1. cities_weather_info.ipynb create from a list of cities a dataframe with weather of this cities.
2. weather_code.py return a dataframe who associate code with a definition of this code.
3. booking_infos.ipynb return the first page of city result for current date.
4. booking_infos_pages.py return the numbers of results availables on firsts pages.
5. booking_hostels.py make a list of all hostels availables for previous list of firts pages.
6. hostels_list.py make a list of all hostels forms from hotels cities list.
7. hostels_list_located.ipynb add locations to this last dataset.

## Informations about merge to create
 * 1 -> cities_weather_infos.csv
 * 2 -> weather_code_description.json : weather_code <-> cities_weather_infos.csv
 * 3 -> booking_mains_cities.csv : city <-> cities_weather_infos.csv
 * 5 -> booking_hostels.json : booking_page <-> +- city : booking_mains_cities.csv
 * 7 -> hostels_list_located.csv : booking_page <-> url_hostel

In [35]:
hostels_list_located = pd.read_csv('src/hostels_list_located.csv')
booking_hostels = pd.read_json('src/booking_hostels.json')
booking_mains_cities = pd.read_csv('src/booking_mains_cities.csv')
weather_code_description = pd.read_json('src/weather_code_description.json')
cities_weather_infos = pd.read_csv('src/cities_weather_infos.csv')

## Merge datasets

In [36]:
# Merge cities_weather_infos & weather_code_description
dataframe = pd.merge(cities_weather_infos, weather_code_description, left_on='weather_code', right_on='weather_code')

# Merge (cities_weather_infos & weather_code_description) & booking_mains_cities
dataframe = pd.merge(dataframe, booking_mains_cities, left_on='city', right_on='city')

# Correct booking_hostels for merge 
booking_hostels['main_url'] = booking_hostels['main_url'].apply(lambda x : re.sub(r'&offset=\d*', '', x))
booking_hostels['OI'] = [np.nan if len(x) == 0 else 1 for x in booking_hostels['url_hostel']]
booking_hostels = booking_hostels.dropna(subset=['OI'])
booking_hostels = booking_hostels.drop(columns=['OI'])

# Merge (cities_weather_infos, weather_code_description, booking_mains_cities) & booking_hostels
dataframe = pd.merge(booking_hostels, dataframe, left_on='main_url', right_on='main_url')

# Correct hostels_list_located & dataframe for merge
hostels_list_located = hostels_list_located.rename(columns={"adress": "hotel_adress",
                                                            "note": "hotel_note",
                                                            "describe": "hotel_describe",
                                                            "latitude": "hotel_latitude",
                                                            "longitude": "hotel_longitude"})
dataframe['url_hostel'] = dataframe['url_hostel'].apply(lambda x : re.sub(r'#hotelTmpl', '', x))

# Final merge with all informations
dataframe = pd.merge(hostels_list_located, dataframe, left_on='booking_page', right_on='url_hostel')

# Clear datas
dataframe = dataframe.drop(columns= ['booking_page', 'main_url'])
dataframe['hotel_note'] = [x.replace(',','.') for x in dataframe['hotel_note']]
dataframe['hotel_note'] = dataframe['hotel_note'].astype(float)

# Save data
dataframe.to_csv(r'src/s3_booking_weather_dataset.csv', index=False)